<a href="https://colab.research.google.com/github/yang0256/Twitter_ETL/blob/main/tweeter_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install transformers
# Load the FinBERT model
from transformers import TFBertForSequenceClassification, BertTokenizer

In [71]:
# tweets collector
import requests
import pandas as pd
import time
# Set the Bearer Token
bearer_token = "AAAAAAAAAAAAAAAAAAAAAN%2FhlQEAAAAAA3KWaOM4ynnWjSNqd%2BH4ncV6mvU%3DRG5HSsW7lgy9kJakzkn67aBPzmfnAHCELtudRJisaeeNUHlkoj"
search_url = "https://api.twitter.com/2/tweets/search/recent"
query_params = {"query": "from:business", "max_results": 100, "tweet.fields": "public_metrics"}

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


# ML

def process_texts(df):
    # Load the pre-trained model and tokenizer
    
    model_name = 'ProsusAI/finbert'
    BertTokenizer.from_pretrained(model_name)
    model = TFBertForSequenceClassification.from_pretrained(model_name)

    # Initialize positive and negative scores
    pro_positive = 0
    pro_negative = 0

    # Loop over the values in the "text" column and aggregate the scores
    for text in df['text']:
        # Tokenize the text
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="tf")

        # Prepare the input
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        # Run inference
        outputs = model(input_ids, attention_mask=attention_mask)
        scores = outputs[0][0].numpy()
        pro_positive += scores[1]
        pro_negative += scores[0]

    return pro_positive, pro_negative



df_scores = pd.DataFrame(columns=['pro_positive', 'pro_negative'])

def main():
    json_response = connect_to_endpoint(search_url, query_params)
    
    tweets_df = pd.DataFrame(json_response['data'])
    #print(tweets_df[0:1])
    print(tweets_df[0:100]['text'])
    
    tweets_df.to_csv("tweets.csv", index=False)

    df = pd.read_csv('tweets.csv')
    
    pro_positive, pro_negative = process_texts(df)

    # add value
    data = {
        'pro_positive': pro_positive,
        'pro_negative': pro_negative
    }
    df_scores.loc[len(df_scores)] = data

if __name__ == "__main__":
    while True:
        main()
        print(df_scores)
        time.sleep(1800) # Sleep for 30 minutes (30 * 60 seconds)
    



KeyboardInterrupt: ignored

In [ ]:
#presentation